# Initial Data Scrape Workbook

***

### We ideally want to scrape a dataset consisting of
- video thumbnails
- titles
- views
- parent channel

### YouTube API
https://developers.google.com/youtube/v3/getting-started#quota

Get credentials by going to https://console.developers.google.com/apis/dashboard?project=red-means-go
- Look for "YouTube Data API v3" in the library tab and make sure it's enabled.
- Select Credentials and get an api key

Daily limit of 10,000 "units" worth of requests.
- Different operations have different cost values, need to be careful what data we request.

We can more efficiently get data by using the offered compressed gzip request format.

In [9]:
# Run once
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

Requirement already up-to-date: google-api-python-client in c:\users\kutor\appdata\local\programs\python\python37\lib\site-packages (1.8.0)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Requirement already up-to-date: google-auth-oauthlib in c:\users\kutor\appdata\local\programs\python\python37\lib\site-packages (0.4.1)
Requirement already up-to-date: google-auth-httplib2 in c:\users\kutor\appdata\local\programs\python\python37\lib\site-packages (0.0.3)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Desired scraping code
- config files to identify what categories of videos to scrape
- what level of popularity to lower bound our videos to
    - what measurement works for this? subscription to yearly average view count in relation to videos uploaded count?
- possible inversion config option to instead opt for getting the least popular videos(?)
- output to data/out/
    - /thumbs -- a folder full of thumbnails with identifying labels (possibly gzip compressed?)
    - videos.csv -- a .csv containing metadata on the videos that correspond to the thumbnails in the above folder.

### Possible search parameters
- Safesearch
    - none
    - moderate
    - strict

***

# Code

In [1]:
import pandas as pd
import numpy as np

import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [6]:
# Input credentials:
from getpass import getpass
api_key = getpass()

········


No idea how the below code is supposed to work. Reference this link to work it out

https://developers.google.com/youtube/v3/docs/search/list?apix_params=%7B%22part%22%3A%22gaming%22%2C%22maxResults%22%3A10%7D

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def execute_request(in_request):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "YOUR_CLIENT_SECRET_FILE.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = eval(in_request)
    response = request.execute()

    return response